### Database using Firestore

Since we only use the read operation, it was easier to generate a database-file to deploy. This code is not used anymore, and may not work.

In [1]:
import kapp_firestore_api as f
import pandas as pd

In [ ]:
csv_files = {
    'babymat': 'babymat.csv',
    'bakeartikler': 'bakeartikler.csv',
    'bakevarer': 'bakevarer.csv',
    'div_matprodukter': 'div_matprodukter.csv',
    'drikke': 'drikke.csv',
    'fisk': 'fisk.csv',
    'frukt_gront': 'frukt_gront.csv',
    'helsekost': 'helsekost.csv',
    'is_dessert': 'is_dessert.csv',
    'meieri_ost': 'meieri_ost.csv',
    'palegg_frokost': 'palegg_frokost.csv',
    'snacks_godteri': 'snacks_godteri.csv',
}

In [ ]:
find_duplicates_all(csv_files)
count_products()

In [ ]:
def read_data(csv_file):
    """
    Local
    Also presents some information about the data.
    Returns: A Panda DataFrame.
    """
    print("------ Reading", csv_file)
    df = pd.read_csv(csv_file)
    return df

In [ ]:
# %%time
def get_docs():
    """
    Firestore
    """
    docs = f.products_ref().get()
    products = []
    for doc in docs:
        products.append(doc.to_dict())
    return products

In [ ]:
def count_products(localOnly = True):
    """
    Firestore & Local
    Count products in CSV files and in the database.
    """
    print("------ Count Products")
    csv_len_sum = 0
    for k,v in csv_files.items():
        df_len = len(pd.read_csv('data/' + v))
        csv_len_sum = csv_len_sum + df_len
        print("{}: {}".format(v, df_len))
    print("---")
    print("All files: {}".format(csv_len_sum))
    if not localOnly:
        print("Database: {}".format(f.nofProducts()))

In [ ]:
def find_duplicates_df(df):
    """
    Local
    Find duplicates in a dataframe (local data from CSV files).
    Duplicates is checked on producer+product.
    """
    print('------ Finding Duplicates')
    dups = df.duplicated(subset=['producer','product'], keep=False) 
    df_dups = df[dups].sort_values(by=['producer','product'])
    display(df_dups)

In [ ]:
def find_duplicates_all(csv_files):
    """
    Local
    See find_duplicates()
    """
    df = pd.DataFrame()
    for k,v in csv_files.items():
        df = pd.concat([df, read_data('data/' + v)], ignore_index=True)
    find_duplicates_df(df)

In [ ]:
def insert_data(df):
    """
    Firestore
    Creates products in the Firebase database.
    Input: A DataFrame
    """
    print("------ Batch Insert Data")
    f.batch_create(df)

In [ ]:
def insert_from_single_file(csv_file):
    """
    Firestore & local
    Insert to Firestore from a single CSV file.
    """
    df = read_data('data/' + csv_file)
    find_duplicates(df)
    # insert_data(df)
    count_products()
    
# insert_from_single_file(csv_files['frukt_gront'])

In [ ]:
def insert_from_files(csv_files):
    """
    Firestore & local
    Insert to Firestore from a set of CSV files.
    """
    for k,v in csv_files.items():
        df = read_data('data/' + v)
#         insert_data(df)

# insert_from_files(csv_files)
# count_products()

In [ ]:
# f.list_products(False)

### Sandbox

In [ ]:
# f.create({
#     u'category': u'REMOVE',
#     u'comment': u'',
#     u'kosher_stamp': u'',
#     u'kosher_type': u'p',
#     u'producer': u'Toro',
#     u'product': u'suppe',
#     u'sub_category': u'',
# })

In [ ]:
# doc_ref1 = f.get_doc_ref(u'-_\n')
# f.pp(doc_ref1)

In [ ]:
# doc_ref1 = f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO')
# doc_ref2 = f.delete(f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO'))
# doc_ref3 = f.get_doc_ref(u'213132')
# f.update(doc_ref3, {u'comment':'Bye'})

In [ ]:
# doc_ref1 = f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO')
# doc_ref2 = f.delete(f.get_doc_ref(u'nTk2VlJoA9MOXxZuMrBs'))

# f.pp(doc_ref1)
# display(f.isDeleted(doc_ref1))
# display(f.exists(doc_ref1))